# 9-10 DAY

In [2]:
import pandas as pd
import numpy as np 

In [3]:
test = pd.read_csv('9D_titanic/test.csv')

In [4]:
train = pd.read_csv('9D_titanic/train.csv')

In [5]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
train_test_dataset = [train, test]

In [8]:
for data in train_test_dataset :
    data['Title'] = data['Name'].str.extract('([a-zA-Z]+)\.')
    data['Title'] = data['Title'].str.lower()
    data['Title'] = np.where(data['Title']== 'mr' , 0,
                            np.where(data['Title']== "mrs" , 1,
                                    np.where(data['Title'] == 'miss', 2 , 3)))
    
    data['Age'] = data['Age'].fillna(data.groupby('Sex')['Age'].transform('median'))
    data['Age'] = np.where(data['Age'] <= 16, 0,
                    np.where(data['Age'] <= 26, 1,
                    np.where(data['Age'] <= 36, 2,
                    np.where(data['Age'] <= 62, 3, 4))))
    
    data['Embarked'].fillna('S', inplace= True)
    data['Embarked'] = data['Embarked'].map({'S':0 , 'C':1, 'Q' :2})
    
    data['Cabin'] = data['Cabin'].str[0]
    cabin_mapping = {'A': 0.0, 'B': 0.4, 'C': 0.8, 'D': 1.2, 'E': 1.6, 'F': 2.0, 'G': 2.4, 'T': 2.8}
    data['Cabin'] = data['Cabin'].map(cabin_mapping)
    data['Cabin'].fillna(data.groupby('Pclass')['Cabin'].transform('median'), inplace=True)
    
    data['Sex'] = data['Sex'].map({'male':0, 'female':1})
    
    data['Fare'].fillna(data.groupby('Pclass')['Fare'].transform('median'),inplace= True)
    data['Fare'] = np.where(data['Fare'] <= 17, 0,
                          np.where(data['Fare'] <= 30, 1,
                                   np.where(data['Fare'] <= 100, 2, 3)))
    
    data['FamilySize'] = data['SibSp'] + data['Parch'] +1
    
    family_map = {1: 0.0, 2: 0.4, 3: 0.8, 4: 1.2, 5: 1.6, 6: 2.0, 7: 2.4, 8: 2.8, 9: 3.2, 10: 3.6, 11: 4.0}
    
    data['FamilySize'] = data['FamilySize'].map(family_map)

In [9]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Title', 'FamilySize'],
      dtype='object')

In [10]:
features_drop = ['PassengerId','Name' , 'Ticket' , 'SibSp' , 'Parch']

In [11]:
train = train.drop(features_drop , axis =1 )

In [12]:
test = test.drop(features_drop , axis =1 )

In [13]:
train.head(1)

,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,FamilySize
0,0,3,0,1,0,2.0,0,0,0.4


In [14]:
test.head(1)

,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,FamilySize
0,3,0,2,0,2.0,2,0,0.0


In [15]:
test.isnull().sum()

Pclass        0
Sex           0
Age           0
Fare          0
Cabin         0
Embarked      0
Title         0
FamilySize    0
dtype: int64

----------

# 11 DAY

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [18]:
inputs = train.drop('Survived', axis =1)
#train.drop(colums='Survived')

In [19]:
outputs = train['Survived']

In [20]:
print(inputs.shape , outputs.shape)
# 891데이터 8 ,속성 891 데이터 1개의 속성 1개라서 표시안됨 

(891, 8) (891,)


In [21]:
outputs.value_counts()
# 0 사망자, 1 생존자 

0    549
1    342
Name: Survived, dtype: int64

In [22]:
train_input, test_input, train_output, test_output = \
train_test_split(inputs, outputs, test_size=0.2, stratify=outputs, random_state=10)
#stratify= a : a라는 파일을 고정 
#random_state : 랜덤값을 고정 

In [23]:
print(train_input.shape, train_output.shape)
print(test_input.shape, test_output.shape)

(712, 8) (712,)
(179, 8) (179,)


In [24]:
train_output.value_counts()

0    439
1    273
Name: Survived, dtype: int64

In [25]:
test_output.value_counts()

0    110
1     69
Name: Survived, dtype: int64

In [26]:
model1 = LogisticRegression()
model2 = DecisionTreeClassifier()
model3 = RandomForestClassifier()
model4 = GradientBoostingClassifier()
model5 = AdaBoostClassifier()

In [27]:
#fit 학습 
model1.fit(train_input, train_output)

LogisticRegression()

In [28]:
# score 결과 
model1.score(train_input, train_output) *100

81.03932584269663

In [29]:
model1.score(test_input, test_output) *100

79.88826815642457

In [30]:
# predict 예측치 
test_pre = model1.predict(test_input)

In [31]:
test_input.iloc[0]

Pclass        3.0
Sex           1.0
Age           0.0
Fare          1.0
Cabin         2.0
Embarked      0.0
Title         2.0
FamilySize    1.6
Name: 24, dtype: float64

In [32]:
test_pre[0]

1

In [33]:
#실제 정답을 보려면 
#iloc 인덱스 
test_output.iloc[0]

0

In [34]:
#하나씩 확인할수 없으니 효율적으로 
accuracy_score(test_pre, test_output) *100

79.88826815642457

In [35]:
model2.fit(train_input, train_output)

DecisionTreeClassifier()

In [36]:
model2.score(train_input, train_output) *100

90.73033707865169

In [37]:
model2.score(test_input, test_output) *100

77.6536312849162

In [38]:
test_pre = model2.predict(test_input)

In [39]:
test_input.iloc[0]

Pclass        3.0
Sex           1.0
Age           0.0
Fare          1.0
Cabin         2.0
Embarked      0.0
Title         2.0
FamilySize    1.6
Name: 24, dtype: float64

In [40]:
test_pre[0]

0

In [41]:
test_output.iloc[0]

0

In [42]:
accuracy_score(test_pre, test_output) *100

77.6536312849162

In [43]:
model = {'LogisticRegression' :LogisticRegression(),
'DecisionTreeClassifier': DecisionTreeClassifier(),
'RandomForestClassifier' : RandomForestClassifier(),
'GradientBoostingClassifier' :  GradientBoostingClassifier(),
'AdaBoostClassifier' : AdaBoostClassifier()}

In [44]:
for m in model :
    clf = model[m]
    clf.fit(train_input, train_output)
    train_score = clf.score(train_input, train_output)*100
    test_score = clf.score(test_input, test_output)*100
    
    print(f"[%26s] train:%.2f / test:%.2f" % (m, train_score, test_score))

[        LogisticRegression] train:81.04 / test:79.89
[    DecisionTreeClassifier] train:90.73 / test:77.65
[    RandomForestClassifier] train:90.73 / test:81.56
[GradientBoostingClassifier] train:86.24 / test:81.56
[        AdaBoostClassifier] train:81.46 / test:82.12


In [45]:
model =  GradientBoostingClassifier()

In [46]:
# fit  학습 
model.fit(train_input, train_output)

GradientBoostingClassifier()

In [47]:
model.score(train_input, train_output) *100

86.65730337078652

In [48]:
model.score(test_input, test_output) *100

83.24022346368714

In [49]:
test_pre = model.predict(test_input)

In [50]:
test_input.iloc[0]

Pclass        3.0
Sex           1.0
Age           0.0
Fare          1.0
Cabin         2.0
Embarked      0.0
Title         2.0
FamilySize    1.6
Name: 24, dtype: float64

In [51]:
test_pre[0]

0

In [52]:
test_output.iloc[0]

0

In [53]:
accuracy_score(test_pre, test_output) *100

83.24022346368714

In [54]:
import pickle

In [55]:
with open('gbc.pk1', 'wb') as f :
    pickle.dump(model, f)
    # 피클 저장?! 

-

In [56]:
with open('gbc.pk1', 'rb') as f :
    load_model = pickle.load(f)
    # 파일 저장을 불러오기 

In [57]:
load_model

GradientBoostingClassifier()

In [58]:
data = test_input.iloc[0].to_list()

In [59]:
data

[3.0, 1.0, 0.0, 1.0, 2.0, 0.0, 2.0, 1.6]

In [60]:
import numpy as np

In [61]:
np_data = np.asarray(data)

In [62]:
re_data = np_data.reshape(1, -1)

In [63]:
pred = load_model.predict(re_data)

C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [64]:
if pred[0] == 0:
    print('Died...')
else :
    print('Servived!')

Died...


-

In [65]:
sub = pd.read_csv('9D_titanic/submission.csv')

In [66]:
answer = sub['Survived']

In [67]:
test.describe()

,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,FamilySize
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,0.363636,1.796651,0.770335,1.687081,0.464115,0.746411,0.335885
std,0.841838,0.481622,0.923599,0.998743,0.563371,0.685516,0.983216,0.607629
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,1.000000,0.000000,1.600000,0.000000,0.000000,0.000000
50%,3.000000,0.000000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000
75%,3.000000,1.000000,2.000000,2.000000,2.000000,1.000000,2.000000,0.400000
max,3.000000,1.000000,4.000000,3.000000,2.400000,2.000000,3.000000,4.000000


In [68]:
load_model

GradientBoostingClassifier()

In [69]:
submission = load_model.predict(test)

In [70]:
accuracy_score(answer,submission) *100

85.64593301435407

In [71]:
sub2= pd.read_csv('9D_titanic/submission2.csv')

In [72]:
sub2['Survived'] = submission

In [73]:
sub2.to_csv('result.csv', index=False)

In [74]:
sub2['Survived'].value_counts()

0    278
1    140
Name: Survived, dtype: int64